In [ ]:
!pip install datasets

# HuggingFace Models Usage and Tokenizer explanation

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

DEVICE = "cuda:0"
MODEL = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForCausalLM.from_pretrained(MODEL).to(DEVICE)

In [ ]:
text = "The sky is"
encoded_input = tokenizer(text, return_tensors='pt').to(DEVICE)
encoded_output = model.generate(**encoded_input, max_new_tokens=5)
output = tokenizer.decode(encoded_output[0], skip_special_tokens=True)

print("encoded_input", encoded_input)
print("encoded_input", encoded_output)
print("OUTPUT: ", output)

### Attention mask

In [ ]:
text = "The most famous city of Italy is "
encoded_input = tokenizer(text, return_tensors='pt').to(DEVICE)
tokens = tokenizer.tokenize(text)
encoded_input.attention_mask[0, 5] = 0

encoded_output = model.generate(**encoded_input, max_new_tokens=16)
output = tokenizer.decode(encoded_output[0], skip_special_tokens=True)

print("tokens", tokens)
print("encoded_input", encoded_input)
print("encoded_output", encoded_output)
print("output: ", output)

### Pipeline usage

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')
output = generator("The sky is", max_new_tokens=5, temperature=0.7, top_k=50, top_p=0.8)

print(output)

# Sentiment Analysis

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

print(dataset)
print(dataset['train'][0])

In [55]:
MODEL = 'google/flan-t5-large'

In [48]:
from datasets import load_dataset
from transformers import pipeline

test_dataset = load_dataset("carblacac/twitter-sentiment-analysis", "test")
train_dataset = load_dataset("carblacac/twitter-sentiment-analysis", "train")

In [ ]:
generator = pipeline("text2text-generation", model=MODEL, device="cuda")

In [ ]:
import random
def build_example(elem):
    return f"""TEXT: {elem['text']}
SENTIMENT: {["NEGATIVE", "POSITIVE"][elem['feeling']]}
"""

def get_build_prompt(dataset, shots=3):
    def build_prompt(sent):
        ret = ""
        for i in range(0, shots):
            ret += build_example(dataset["train"][i])+"\n"
        ret += f"TEXT: {sent}\n"
        ret += "SENTIMENT:"

        return ret

    return build_prompt

prompt = get_build_prompt(train_dataset, shots=3)
print(prompt("This is my sentence"))

In [51]:
from transformers import pipeline

def inference(*, prompt, sentence, generator):
    input = prompt(sentence)
    generated = generator(input, max_new_tokens=5, pad_token_id=generator.tokenizer.eos_token_id)[0]['generated_text']
    generated = generated.replace(input, "").strip()
    output = generated.split("\n")[0]

    if "NEGATIVE" in output:
        return 0
    elif "POSITIVE" in output:
        return 1
    else:
        print("Model is confused... generated text: ", generated)
        return 0

print(inference(prompt=prompt, sentence="This is so bad", generator=generator))

0


In [60]:
import torch
from tqdm import tqdm
from transformers import pipeline

@torch.no_grad()
def eval():
    total = 200
    targets = []
    predictions = []

    for i, elem in tqdm(enumerate(test_dataset['test']), total=total):
        targets.append(elem['feeling'])
        predictions.append(inference(prompt=prompt, sentence=elem['text'], generator=generator))
        if i >= total:
            break

    return targets, predictions

In [ ]:
results = eval()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(results[0], results[1]))

## Feature extraction

In [ ]:
feature_extractor = pipeline("feature-extraction", framework="pt", model="bert-base-uncased")
text = "This is a sentence"

features = feature_extractor(text,return_tensors = "pt")[0].numpy()   # (6, 768)
mean_features = features.mean(axis=0)

In [16]:
import numpy as np

def get_feature_extractor(model):
    if model == "bert":
        pipe = pipeline("feature-extraction", framework="pt", model="bert-base-uncased", device="cuda")

        def feature_extractor(text):
            features_full = pipe(text)
            features = []
            for full in features_full:
                full = np.array(full[0])
                features.append(full.mean(axis=0))
            return np.array(features)

        return feature_extractor, 768

In [ ]:
from datasets import load_dataset

MODEL = "bert"

extractor, feature_size = get_feature_extractor(MODEL)
test_dataset = load_dataset("carblacac/twitter-sentiment-analysis", "test")
train_dataset = load_dataset("carblacac/twitter-sentiment-analysis", "train")

In [ ]:
from tqdm import tqdm

test_total = 100
test_features = []
test_classes = []
for i, elem in tqdm(enumerate(test_dataset['test']), total=test_total):
    test_classes.append(elem['feeling'])
    test_features.append(extractor([elem['text']])[0])
    if i >= test_total:
        break

train_total = 1000
train_features = []
train_classes = []
for i, elem in tqdm(enumerate(train_dataset['train']), total=train_total):
    train_classes.append(elem['feeling'])
    train_features.append(extractor([elem['text']])[0])
    if i >= train_total:
        break

train_features = np.array(train_features)
test_features = np.array(test_features)

print("train", train_features.shape)
print("test", test_features.shape)

In [ ]:
from tqdm import tqdm

total = 1000

features = []
classes = []
for i, elem in tqdm(enumerate(test_dataset['test']), total=total):
    classes.append(elem['feeling'])
    features.append(extractor([elem['text']])[0])
    if i >= total:
        break

features = np.array(features)
print(features.shape)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

tsne = TSNE(n_components=2)
features_2d = tsne.fit_transform(features)

plt.title(MODEL)
plt.scatter(features_2d[:, 0], features_2d[:, 1], c=classes, s=5)
plt.show()

In [21]:
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier


classifiers = dict(
    lr=LogisticRegression(max_iter=10000),
    mlp=MLPClassifier(max_iter=10000),
    knn=KNeighborsClassifier(n_neighbors=5)
)

reports = dict()

for name, clf in classifiers.items():
    clf.fit(train_features, train_classes)
    predictions = clf.predict(test_features)
    reports[name] = classification_report(test_classes, predictions, output_dict=True)

In [ ]:
print("name\tacc\tf1")
for name, report in reports.items():
    print(f"{name}\t{report['accuracy']:.2f}\t{report['weighted avg']['f1-score']:.2f}")

# Sentence Transformers

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

sentences = [
    "I think dogs are the best pets.",
    "I think cats are the best pets.",
    "Deep learning is a subfield of machine learning.",
]

embeddings = model.encode(sentences)

In [ ]:
import torch

print(embeddings.shape)
embeddings = torch.tensor(embeddings)

cosine_scores = torch.nn.functional.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=2)
print(cosine_scores)

# Fill Mask

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

# Generate Keywords

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")


In [ ]:
llm_wikipedia = "A large language model (LLM) is a type of artificial intelligence (AI) algorithm that uses deep learning techniques and massively large data sets to understand, summarize, generate and predict new content. The term generative AI also is closely connected with LLMs, which are, in fact, a type of generative AI that has been specifically architected to help generate text-based content."

inputs = tokenizer(f"Generate keywords for this text:\n{llm_wikipedia}\nkeywords: ", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50, top_p=0.8, top_k=50, temperature=0.7)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

# Few Shot Prompting

In [29]:
MODEL = "flan-t5"

In [ ]:
from datasets import load_dataset

test_dataset = load_dataset("carblacac/twitter-sentiment-analysis", "test")
train_dataset = load_dataset("carblacac/twitter-sentiment-analysis", "train")


def label_to_text(label):
    label = int(label)
    if label == 0:
        return "NEGATIVE"
    elif label == 1:
        return "POSITIVE"
    else:
        raise Exception("Unknown label")

def text_to_label(text):
    if "NEGATIVE" in text:
        return 0
    elif "POSITIVE" in text:
        return 1
    else:
        print(f"ERROR: unknown label {text}")
        return 0

In [31]:
def get_model(shots=3):
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to("cuda")
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

    def inference(text):
        def encode_example(elem):
            return f"text: {elem['text']}\nsentiment: {label_to_text(elem['feeling'])}"
        prompt = f"Classify the sentiment of the following texts in one of these two categories: POSITIVE, NEGATIVE.\n\n"
        for i in range(0, shots):
            prompt += encode_example(train_dataset["train"][i])+"\n"
        prompt += f"text: {text}\nsentiment:"

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=50)
        return text_to_label(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

    return inference


In [ ]:
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report

overall_results = dict()

for shots in [0, 5, 10]:
    print("Running shot", shots)
    model = get_model(shots=shots)

    total = 100
    targets = []
    predictions = []

    for i, elem in tqdm(enumerate(test_dataset['test']), total=total):
        targets.append(elem['feeling'])
        predictions.append(model(elem['text']))
        if i >= total:
            break


    report = classification_report(targets, predictions, output_dict=True)
    overall_results[shots] = report

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

dfs = []

for shot, report in overall_results.items():
    dfs.append(dict(
        shot=shot,
        accuracy=report["accuracy"],
        **report["macro avg"]
    ))

df = pd.DataFrame(dfs)
print(df)
plt.plot(df["shot"], df["precision"], c="k")
plt.plot(df["shot"], df["recall"], c="r")
plt.plot(df["shot"], df["f1-score"], c="b")
#plt.scatter(df["shot"], df["support"], c="g")

plt.show()

In [ ]:
plt.plot(df["shot"], df["accuracy"])

# Traslator

In [ ]:
!pip install evaluate

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("opus_books", "en-it", split="train[1%:2%]")
val_dataset = load_dataset("opus_books", "en-it", split="train[2%:100%]")

In [ ]:
print(train_dataset)
print(val_dataset)

In [ ]:
for i, elem in enumerate(train_dataset):
    print(elem)
    if i >= 5:
        break

In [7]:
def get_translator():
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to("cuda")
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

    def translator(text):
        def encode_example(elem):
            return f"text: {elem['translation']['en']}\ntranslation: {elem['translation']['it']}\n"
        prompt = "Translate this text from English into Italian.\n\n"
        for i, elem in enumerate(train_dataset):
            prompt += encode_example(elem)+"\n"
            if i >= 5:
                break
        prompt += f"text: {text}\ntranslation:"

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=50)
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    return translator


In [ ]:
translator = get_translator()
print(translator("Hello, how are you?"))

In [ ]:
from tqdm import tqdm

count = 100
predicted = []
actual = []

for i, elem in enumerate(tqdm(val_dataset, total=count)):
    if i >= count:
        break

    text = elem["translation"]["en"]
    translation = elem["translation"]["it"]
    prediction = translator(text)
    predicted.append(prediction)
    actual.append(translation)


In [ ]:
import pandas as pd

pd.DataFrame(dict(actual=actual, predicted=predicted))

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
bleu.add_batch(predictions=predicted, references=actual)
results = bleu.compute()
print(results)